In [2]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


# numericalization
from collections import Counter

# preprocessing
import re
import nltk
from nltk.corpus import stopwords # will give an altered version later cuz the default isn't great
from string import punctuation
# from textblob import TextBlob
from collections import Counter
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

# modeling
from sklearn.model_selection import train_test_split

# neural nets
import tensorflow as tf
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras import Sequential, Input, optimizers
from keras.optimizers import Adam
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer, BertForSequenceClassification, 
    BertTokenizer, AutoModelForSequenceClassification, AutoTokenizer, AdamW
)

pd.set_option('display.max_columns', 500)
title_fontsize = 15

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/setone/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/setone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/setone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/setone/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/Users/setone/miniforge3/envs/tf-gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('./data/large_datafiles/all_stock_score.csv').drop_duplicates()
stocks = pd.read_csv('./data/large_datafiles/scraped_stock_2015_2022.csv').iloc[:, 1:]

In [69]:
# df.to_csv('./data/large_datafiles/all_tweets_2015_2022.csv')
# df = df.iloc[:, 1:]

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('./data/model')
model = RobertaForSequenceClassification.from_pretrained('./data/model')

In [5]:
def func(x):
    if x=='negative':
        return -1
    elif x=='positive':
        return 1
    else:
        return 0

df['label'] = df['label'].apply(lambda x: func(x))

In [6]:
df = df[~df['label'] == 0]
df

,Unnamed: 0,Date,Content,ticker,score,label
1,1,2015-01-01 00:01:36,Insanity of today weirdo massive selling. $aap...,AAPL,-0.8271,-1
38,38,2015-01-01 00:49:05,Jeff Bezos lost $7.4 billion in Amazon's worst...,AMZN,-0.7506,-1
39,39,2015-01-01 00:50:10,Jeff Bezos lost $7.4 billion in #Amazon worst ...,AMZN,-0.6908,-1
40,40,2015-01-01 00:50:21,Ouch. RT @WSJ: Jeff Bezos lost $7.4 billion in...,AMZN,-0.7506,-1
41,41,2015-01-01 00:50:56,And I bet he has billions more! “@WSJ: Jeff Be...,AMZN,-0.7712,-1
...,...,...,...,...,...,...
4579069,6201666,2015-10-26 18:23:55,"Lies, damn lies &amp; statistics: the ""process...",ZS,-0.8957,-1
4579095,6201692,2015-10-16 02:13:04,It turns out that IFTTT's Twitter integration ...,ZS,-0.7607,-1
4579102,6201699,2015-10-08 04:52:56,The Light L16 is likely to upset you if you ha...,ZS,-0.7003,-1
4579215,6201812,2015-06-17 22:43:02,@speak_io @particle Unfortunately not the best...,ZS,-0.7236,-1


In [7]:
df = df.merge(stocks, left_on='ticker', right_on='Stock Name')